In [6]:
import pandas as pd
import importlib
import fileDistanceProcessing as fdp
import importlib
importlib.reload(fdp) 

<module 'fileDistanceProcessing' from '/home/marko/Desktop/ip2/ip2_projekat/source/fileDistanceProcessing.py'>

In [7]:
X=pd.read_csv("../data/csv/sars2_mers_sars1.txt",skipinitialspace = True )

In [8]:
X.columns = X.columns.str.strip()
X.columns

Index(['Virus', 'ACCESSION', 'WHO_TYPE', 'ACCESSION_PROT', 'PROTEIN',
       'DUZINA_NUC', 'KOD_NUC'],
      dtype='object')

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [10]:
import pickle
import os
from zipfile import ZipFile

#raspakuj arhivu
with ZipFile("../data/full_distances_matrix.zip",'r') as zip:
    zip.extractall("..")

In [11]:
# upcitaj matricu distanci
with open("../data/full_distances_matrix", "rb") as infile:
     	distances = pickle.load(infile)

os.remove("../data/full_distances_matrix")

In [12]:
# metrika zasnovana samo na 5-adic distanci
# instant racunanje iz matrice
def five_adic_distance_from_matrix(x, y):
    
    i = int(x[-1]) #indeks je na poslednjem mestu u listi
    j = int(y[-1])

    # matrica je u obliku trougla bez dijagonale originalne matrice
    first = min(i, j)
    second = max(i, j) - (first + 1) # veci indeks smanji za broj instanci u redu matrice do dijagonale 

    return 0 if i == j else distances[first][second] #udaljenost x od y

In [13]:
y_virus=X["Virus"]
X_virus=[] # potreban nam je samo indeks

# na svaku instancu X_virus smo appendovali njen index
for i in range(len(y_virus)):
    X_virus.append([0, i]) # 0 je dodata zbog knn da ne pravi problem
    
X_train, X_test, y_train, y_test = train_test_split(X_virus, y_virus, test_size=0.33, stratify=y_virus)

In [14]:
from sklearn.metrics import accuracy_score, confusion_matrix

def perform_and_evaluate_knn(k, metric):
    model = KNeighborsClassifier(n_neighbors=k, algorithm='ball_tree', metric=metric)
    model.fit(X_train, y_train)

    y_pred=model.predict(X_test)
    print(f'model evalutaion for k={k} and metric={str(metric)}')
    print(f'accuracy_score: {accuracy_score(y_test,y_pred)}')
    print(confusion_matrix(y_test,y_pred))

In [15]:
perform_and_evaluate_knn(3, five_adic_distance_from_matrix)

model evalutaion for k=3 and metric=<function five_adic_distance_from_matrix at 0x7f85d4a04ca0>
accuracy_score: 0.9988524213908653
[[ 610    0    1]
 [   0    2    4]
 [   0    0 3740]]


In [16]:
y_protein = X["PROTEIN"]
X_protein = X_virus

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_protein, y_protein, test_size=0.33, stratify=y_virus)

In [18]:
perform_and_evaluate_knn(3, five_adic_distance_from_matrix)

model evalutaion for k=3 and metric=<function five_adic_distance_from_matrix at 0x7f85d4a04ca0>
accuracy_score: 0.9995409685563461
[[1405    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0]
 [   0 1878    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0]
 [   0    0    1    0    0    0    0    0    0    0    0    0    0    0
     0    0    0]
 [   0    0    0   43    0    0    0    0    0    0    0    0    0    0
     0    0    0]
 [   0    0    0    0   65    0    0    0    0    0    0    0    0    0
     0    0    0]
 [   0    0    0    0    0   18    0    0    0    0    0    0    0    0
     0    0    0]
 [   0    0    0    0    0    0   38    1    0    0    0    0    0    0
     0    0    0]
 [   0    0    0    0    0    0    1   39    0    0    0    0    0    0
     0    0    0]
 [   0    0    0    0    0    0    0    0    3    0    0    0    0    0
     0    0    0]
 [   0    0    0    0    0    0    0    0    0    9    0   

In [19]:
import numpy as np

virusTypes = np.array(X['Virus'])
X_who = []
indexes = []
for i in range(len(virusTypes)):
    if(virusTypes[i].strip() == "SARS_COV_2"):
        X_who.append([0, i])
        indexes.append(i)

sars2_only = X.filter(items=indexes, axis=0)

In [20]:
y_who=sars2_only["WHO_TYPE"].str.strip()

X_train, X_test, y_train, y_test = train_test_split(X_who, y_who, test_size=0.33, stratify=y_who)

In [21]:
perform_and_evaluate_knn(3, five_adic_distance_from_matrix)

model evalutaion for k=3 and metric=<function five_adic_distance_from_matrix at 0x7f85d4a04ca0>
accuracy_score: 0.9657754010695188
[[478   3   1   3   0   1   2   0   0   0   0   0   7]
 [  0 186   0   1   0   0   3   0   0   0   1   0   2]
 [  4   1 478   1   0   0   2   2   0   0   2   0   5]
 [  0   5   1 485   0   0   1   0   0   1   2   0   0]
 [  3   2   0   0 197   1   0   0   0   0   1   0   3]
 [  3   1   0   4   0 481   1   0   0   0   4   1   0]
 [  0   0   1   2   0   1 488   0   0   0   3   0   0]
 [  1   1   2   0   0   0   1  47   0   0   0   0   0]
 [  0   1   1   0   0   1   2   0 127   0   0   0   3]
 [  1   2   0   2   0   0   1   0   0  16   0   0   0]
 [  7   3   6   2   0   0   3   0   1   0 473   0   0]
 [  0   0   1   0   0   0   0   0   0   0   1   5   0]
 [  0   1   2   0   0   0   0   0   0   0   0   0 151]]
